In [1]:

from config import GlobalConfig, tokenizer_parse_args
from helpers.Data import TokensPaths, load_queries, load_dictionary
from transformers import AutoTokenizer
import json


c:\Users\antoi\Desktop\thesis\biosyn_faiss_prod\myenc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from tokenizer import tokenize_names

tokenizer = AutoTokenizer.from_pretrained(cfg.model.model_name, use_fast=True)
tokenizer.add_special_tokens(cfg.tokenize.special_tokens)


# tokenize_names(queries_sentences, 
#                 tokens_paths.queries_input_ids_path, 
#                 tokens_paths.queries_attention_mask_path, 
#                 max_length=queries_max_length, batch_size=cfg.tokenize.tokenize_batch_size)
names= queries_sentences
N = len(names)
N


10

In [7]:

tokenized = tokenizer(names, padding="max_length", max_length=queries_max_length, truncation=True)
tokenized


{'input_ids': [[101, 170, 16358, 3702, 12733, 1104, 1103, 28996, 8050, 26601, 21943, 2285, 185, 23415, 5674, 4863, 1884, 2646, 189, 27226, 28997, 17203, 1766, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1103, 28996, 8050, 26601, 21943, 2285, 185, 23415, 5674, 4863, 1884, 2646, 113, 170, 1643, 1665, 114, 189, 27226, 28997, 17203, 1766, 4592, 7451, 1103, 192, 2227, 25498, 13548, 1118, 5071, 170, 2703, 1114, 176, 1193, 2528, 4915, 188, 26588, 6530, 24779, 124, 16632, 1161, 113, 102], [101, 1107, 28996, 1884, 4934, 1610, 16430, 7903, 28997, 3652, 117, 2445, 1104, 170, 1643, 1665, 4501, 1106, 1103, 23168, 1104, 11933, 20127, 10430, 1107, 1103, 14297, 117, 1187, 1122, 23126, 1106, 1105, 23162, 1116, 1103, 189, 1665, 2087, 118, 125, 15416, 5318, 113, 7815, 1107, 164, 102], [101, 1112, 7160, 1606, 14715, 9080, 15416, 1348, 14915, 3919, 22979, 1107, 170, 1643, 1665, 118, 120, 118, 28996, 1884, 4934, 1610, 16430, 7903, 28997, 3652, 119, 102, 0, 0, 0, 0, 

In [8]:
mention_start_token_id  = tokenizer.convert_tokens_to_ids(cfg.tokenize.special_tokens_dict["mention_start"])
mention_end_token_id  = tokenizer.convert_tokens_to_ids(cfg.tokenize.special_tokens_dict["mention_end"])
mention_start_token_id, mention_end_token_id 

(28996, 28997)

In [16]:
names[2]

np.str_('In [MS] colon carcinoma [ME] cells , loss of APC leads to the accumulation of betacatenin in the nucleus , where it binds to and activates the Tcf - 4 transcription factor ( reviewed in [ 1 ] [ 2 ] )')

In [40]:
import torch
l = torch.tensor(input_ids == mention_start_token_id).nonzero(as_tuple=True)[0]
l.device

device(type='cpu')

In [25]:
import numpy as np
input_ids = np.array(tokenized["input_ids"])
(input_ids == mention_start_token_id).nonzero()[1]

array([ 7,  2,  2, 18, 17, 15,  6, 25, 35, 11])

In [18]:
import numpy as np
input_ids = np.array(tokenized["input_ids"])
mention_start_positions = (input_ids == mention_start_token_id).nonzero(as_tuple=True)[0]
mention_end_positions = (input_ids == mention_end_token_id).nonzero(as_tuple=True)[0]

inps[2], mention_start_positions , mention_end_positions

TypeError: ndarray.nonzero() takes no keyword arguments

In [ ]:
embeding queries cuis: ['C0010674' 'C0010674']